# Image Transformations
*Author: Vladislav Kim*
* [Introduction](#intro)
* [Contrast adjustment and image inversion](#contrast)
* [Denoising](#noise)
* [Thresholding: separate foreground from background](#threshold)
* [Morphological operations](#morphology)
* [Non-uniform illumination correction](#bgcorrect)


<a id="intro"></a> 
## Introduction

Before applying segmentation or training a machine learning model one may have to transform raw images and adjust numerous parameters such as brightness, contrast, noise level. We may also want to combine or split color channels or apply filters that enhance or suppress certain image features. In this notebook we will show a number of preprocessing techniques that may come handy in the context of microscopy data.

In [ ]:
# load third-party Python modules
import javabridge
import bioformats as bf
import skimage
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

import sys
sys.path.append('..')

javabridge.start_vm(class_path=bf.JARS)

We will start by loading the same image of stained nuclei:

In [ ]:
from base.utils import read_image
img_ho = read_image(fname='data/CLL-coculture/r01c02f01-Hoechst.tiff')

Images, internally represented as numerical arrays, can have either integer-valued ('uint8', 'uint16') or continuous ('float') pixel intensities. You can learn more about image data types [here](https://scikit-image.org/docs/dev/user_guide/data_types.html). Some functions require images to be of type 'float' and you can use `skimage.img_as_float` function to convert an integer-valued image to a float representation. Note that this image is of type 'float'.

In [ ]:
img_ho.dtype

<a id="contrast"></a> 
## Contrast adjustment and image inversion
It is not uncommon for raw microscopy images be somewhat too dark. To adjust brightness of an image, one may have to apply gamma correction to the image, which simply raises the image represented as `np.array` to the power of `gamma`

In [ ]:
# if the numpy.array is of dtype 'float'
gamma = 0.3
img_gamma = img_ho ** gamma

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_gamma, cmap='gray')
plt.axis('off')

Power transformation (with 0 < `gamma` < 1) shifts pixel intensities to higher values and thus increases brightness. The effect of the transformation can be further assessed by comparing the histograms of the original and power-transformed (gamma-corrected) images:

In [ ]:
plt.figure(figsize=(6,6))
sn.distplot(img_ho, kde=False, label='Original image')
sn.distplot(img_gamma, kde=False, label='Gamma-corrected')
plt.xlabel('Intensity')
plt.legend()

In some cases gamma correction is not sufficient and more sophisticated methods such as adaptive histogram equalization (CLAHE) should be used to adjust the contrast of raw images:

In [ ]:
from skimage.exposure import equalize_adapthist
img_adj = equalize_adapthist(img_ho)

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(img_adj, cmap='gray')
plt.axis('off')

In [ ]:
plt.figure(figsize=(6,6))
sn.distplot(img_ho, kde=False, label='Original image')
sn.distplot(img_adj, kde=False, label='CLAHE-adjusted')
plt.xlabel('Intensity')
plt.legend()

Note that adaptive histogram equalization (`equalize_adapthist`) does not shift the peak of the image histogram, but instead spreads out the distribution of intensity values. This usually results in local contrast enhancement, but histogram equalization may also magnify noise.


We will continue working with the gamma-corrected image of the nuclei. Perhaps the most trivial image transformation is image inversion (obtaining the "negative" of the image). Given input image `img_gamma` of 'float' data type we can obtain the complement: `1.0 - img_gamma`. In the inverted (negative) image the background is bright and the nuclei appear dark

In [ ]:
# if the numpy.array is of dtype 'float'
img_neg = 1. - img_gamma

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_neg, cmap='gray')
plt.axis('off')

In [ ]:
plt.figure(figsize=(6,6))
sn.distplot(img_gamma, kde=False, label='Original image')
sn.distplot(img_neg, kde=False, label='Negative image')
plt.xlabel('Intensity')
plt.legend(loc='upper center')

A useful feature is image rescaling, which can be achieved by `skimage.transform.resize`:

In [ ]:
# dimensions of the transformd image
img_neg.shape

Using `resize` we can downsample the original image which has dimensions 2160 x 2160 and convert it to a much smaller image (200 x 200):

In [ ]:
from skimage.transform import resize
img_small = resize(img_neg, (200, 200), anti_aliasing=False)

Up- or downsampling images may introduce artefacts ('aliasing'). If we view the resized image we can see that it's much more pixelated than the original:

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(img_small, cmap='gray')

<a id="noise"></a> 
## Denoising
Consider the resized negative image: by downsampling it from the original size of (2160 x 2160) to (200 x 200) pixels we introduced some image artefacts. A common approach to address this issue is anti-aliasing which in its simplest form applies Gaussian filter to smooth aliasing effects.

In [ ]:
from skimage.filters import gaussian
img_smooth = gaussian(img_small, sigma=0.55)

In [ ]:
from base.plot import plot_channels
plot_channels([img_small, img_smooth], nrow=1, ncol=2, 
              cmap='gray',
              titles = ['Downsampled (200x200)', 'Smoothed (200 x 200)'],
             scale_x=6, scale_y=6)

After applying Gaussian blur the image appears smoother. Gaussian filter is a common tool for smoothing images. The value of `sigma` has to be tweaked  manually and is usually chosen "by eye". Note that there are limits to image denoising and due to almost 10-fold downsampling restoring the original quality would be challenging with simple spatial-domain filters.


Depending on the noise model we may have to adapt our denoising strategy. Let's simulate the following noise models:
+ Gaussian
+ Poisson
+ Salt-and-pepper noise
+ Speckle noise

In [ ]:
from skimage.util import random_noise
gauss_noisy = random_noise(img_neg, mode='gaussian', var=0.01, seed = 3)
poisson_noisy = random_noise(img_neg, mode='poisson', seed = 3)
salt = random_noise(img_neg, mode='salt', seed = 12)
pepper = random_noise(img_neg, mode='pepper', seed = 3)
speckle = random_noise(img_neg, mode='speckle', seed=1)

noise_models = ['Gaussian', 'Poisson', 'Salt',
                'Pepper', 'Speckle']

In [ ]:
plot_channels([gauss_noisy, poisson_noisy, salt, pepper, speckle],
              nrow=2, ncol=3,
              titles=noise_models,
              scale_x=6, scale_y=6, 
              cmap='gray')
plt.tight_layout()

For salt-and-pepper noise a median filter may be sufficient to denoise the image:

In [ ]:
from skimage.filters import median

plot_channels([median(salt), median(pepper)],
              nrow=1, ncol=2,
              titles=['Median-filtered salt noise', 'Median-filtered pepper noise'],
              scale_x=6, scale_y=6,
              cmap='gray')

For Gaussian and speckle noise we can use variational denoising

In [ ]:
from skimage.restoration import estimate_sigma, denoise_tv_chambolle, denoise_nl_means
sigma_est = estimate_sigma(gauss_noisy)

In [ ]:
plot_channels([denoise_tv_chambolle(gauss_noisy, weight=0.3),
               denoise_tv_chambolle(speckle, weight=0.2)],
              titles=['Gaussian noise reduced by TV Chambolle', 'Speckle noise reduced by TV Chambolle'],
              scale_x=6, scale_y=6,
              nrow=1, ncol=2, cmap='gray')

There is a number of other variational (Bregman total variation, non-local means) and wavelet-transform based denoising methods implemented in `skimage.restoration` module. Note that most of these methods are computationally intensive, e.g. non-local means denoising (in `fast_mode=False`) on 512 x 512 image takes:

In [ ]:
# this code chunk runs very slowly
# uncomment to see how much time it takes
'''%%timeit
denoise = denoise_nl_means(gauss_noisy[:512,:512],
                           h=0.8 * sigma_est, 
                           fast_mode=False,
                           patch_size=10,
                           patch_distance=10,
                           multichannel=False)'''

You can read up more on spatial-domain filters in `skimage` [documentation](https://scikit-image.org/docs/dev/api/skimage.filters.html). Variational and wavelet-based denoising techniques are described [here](https://scikit-image.org/docs/dev/api/skimage.restoration.html). 

<a id="threshold"></a>
## Thresholding: separate foreground from background
One of the simplest yet useful "classification" tasks in image processing is thresholding: sepration of foreground from background pxiels. Most of the classical (non-machine learning) techniques rely on setting a cutoff (threshold) in an image histogram based on a certain criterion.

We will use the image with nuclei and try to identify foreground pixels. You can try all available thresholding methods in `skimage` at once by using `try_all_threshold`:

In [ ]:
from skimage.filters import try_all_threshold

fig, ax = try_all_threshold(img_gamma, figsize=(8,16), verbose=False)
plt.tight_layout()

A thresholded image is binarized: foreground pixels are white (`1`'s) and the background is black (`0`'s). Note that some methods are extremely conservative: Yen and minimum thresholding remove all large low-intensity nuclei and leave only few bright spots. Mean thresholding on the other hand retains all low-intensity nuclei (and some noise), while Otsu and triangle methods are somehwat in-between.

Note that we applied thresholding to the power-transformed image `img_gamma`. Had we used the  raw image `img_ho` instead, the results would have been different (uncomment the following code block to see the result)

In [ ]:
'''fig, ax = try_all_threshold(img_gamma, figsize=(8,16), verbose=False)
plt.tight_layout()'''

Based on `try_all_threshold` output one could for example choose Otsu method to separate foreground from background. Function `threshold_otsu` returns the cutoff value for foreground pixels:

In [ ]:
from skimage.filters import threshold_otsu
th = threshold_otsu(img_gamma)


sn.distplot(img_gamma, kde=False, label='Image histogram')
plt.axvline(th, color='red', label='Otsu threshold')
plt.legend()

Threshold the image based on the obtained value:

In [ ]:
img_th = img_gamma.copy()
img_th[img_th < th] = 0
img_th[img_th >= th] = 1

The best is always to compare the original and transformed image side-by-side:

In [ ]:
plot_channels([img_gamma, img_th], nrow=1, ncol=2,
             titles=['Image', 'Thresholded by Otsu method'],
             cmap='gray')

<a id="morphology"></a> 
## Morphological operations
Some image preprocessing steps such as thresholding may introduce discontinuities (e.g. holes inside nuclei) in transformed images or merge objects that are close to one another. We can address such issues using morphological operations which can alter shape, size or connectivity in binarized images.


Some of the most common morphological opearations are:
+ dilation
+ erosion
+ opening
+ closing

We can apply these operations that are available in `skimage.morphology` module to our thresholded image:

In [ ]:
# the thresholded image is binary
print(np.unique(img_th))

In [ ]:
# specify neighborhood element
from skimage.morphology import disk
selem = disk(10)

In [ ]:
from skimage.morphology import binary_dilation, binary_erosion, binary_closing, binary_opening

In [ ]:
dilated = binary_dilation(img_th, selem)
eroded = binary_erosion(img_th, selem)
opened = binary_opening(img_th, selem)
closed = binary_closing(img_th, selem)

In [ ]:
morphs = ['Original', 'Dilation', 'Erosion',
         'Opening', 'Closing']

In [ ]:
plot_channels([img_th, dilated, eroded, opened, closed],
              nrow=2, ncol=3,
              titles=morphs,
              scale_x=6, scale_y=6, 
              cmap='gray')
plt.tight_layout()

<a id="bgcorrect"></a>
## Non-uniform illumination correction
Occasionally micrsocopy images may have non-uniform illumination: e.g. one side of the image appears darker due to uneven illumination. We can correct such issues using background subtraction. 

Let's load an image of leukemia cells stained with surface marker antibody:

In [ ]:
img = read_image(fname="data/BiTE/Tag1r02c02-APC.tiff")
mip = img**0.5

If we appply thresholding directly to this image, we will see that some methods overestimate the number of foreground pixels in the lower right corner due to non-uniform illumination (which is not easy to spot by simply looking at the original image)

In [ ]:
fig, ax = try_all_threshold(mip**0.5, figsize=(8,16), verbose=False)
plt.tight_layout()

We can estimate the background and subtract it from the original image in order to correct non-uniform illumination. The most straightforward way to estimate the background of an image is to apply a Gaussian blur with a large $\sigma$:

In [ ]:
# estimate background using large 'sigma'
bg = gaussian(mip, sigma=200)

In [ ]:
# original with background subtracted
bgsub = mip - bg

In [ ]:
titles = ['Original', 'Estimated background ($\sigma=200$)', 'Background-subtracted']
plot_channels([mip, bg, bgsub],
              ncol=3,nrow=1,
              cmap='gray',
              titles=titles,
              scale_x=6, scale_y=6)

After background subtraction Li and mean thresholding becomes more uniform and less fixated on the lower-right corner 

In [ ]:
fig, ax = try_all_threshold(bgsub, figsize=(8,16), verbose=False)
plt.tight_layout()